# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create keyspace
try:
    session.execute("Create keyspace IF NOT EXISTS event WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1}")
except Exception as ex:
    print(f"Error occured in creating keyspace{str(ex)}")

#### Set Keyspace

In [7]:
# Set the newly created keyspace as default keyspace for current session
try:
    session.set_keyspace("event")
except Exception as ex:
    print(f"Error occured in setting keyspace{str(ex)}")

#### In order to find details for a particular sessionId & itemInSession we need to create a table with primary key sessionId & clustering column itemInSession

##### Lets call the table session_song with column sessionId, artist, song_title, song_length with the aforesaid primary key & clustering columns.
                   

In [8]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS session_song (session_id int, item_in_session int, artist Text, song_title Text, song_length float, PRIMARY KEY (session_id, item_in_session))""")
    print("Table session_song created successfully!!")
except Exception as ex:
    print(f"Error creating table {str(ex)}")

Table session_song created successfully!!


In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    count = 0
    print("Loading data to cassandra")
    for line in csvreader:
        # Prepare insert query
        query = "Insert into session_song (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        # Break line elements into individual variables to use in insert statement
        artist, first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id = line
        try:
            session.execute(query, (int(session_id), int(item_in_session),artist,song, float(length)))
            count += 1
        except Exception as ex:
            count += 1
            print(f"Error inserting row #{count} {str(ex)}")
print("Data loading completed")

Loading data to cassandra
Data loading completed


#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
print("Executing select query to test the data model as per given query")
res = session.execute("select artist,song_title,song_length from session_song where session_id = 338 and item_in_session = 4")

# Add header for the output row

print("artist_name\tsong_title\tsong_length")
for r in res:
    print(r.artist,"\t" + r.song_title + "\t" + str(r.song_length))

Executing select query to test the data model as per given query
artist_name	song_title	song_length
Faithless 	Music Matters (Mark Knight Dub)	495.30731201171875


###### In order to find details for a particular userId & sessionId we need to create a table with primary key sessionId & userId and to sort with itemInSession that will become its clustering column.

###### Lets call the table user_session with column artist, song_title, first_name, last_name with the aforesaid primary key & clustering columns.

In [11]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS user_session (user_id int, session_id int, item_in_session int, artist Text, song_title Text, first_name Text, last_name Text, PRIMARY KEY ((user_id, session_id), item_in_session))""")
    print("Table user_session created successfully!!")
except Exception as ex:
    print(f"Error creating table {str(ex)}")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    count = 0
    print("Loading data in cassandra")
    for line in csvreader:
        query = "Insert into user_session (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
        # Break line elements into individual variables to use in insert statement
        artist, first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id = line
        try:
            session.execute(query, (int(user_id),int(session_id), int(item_in_session), artist,song, first_name, last_name))
            count += 1
        except Exception as ex:
            count += 1
            print(f"Error inserting row #{count} {str(ex)}")
print("Data loading completed")
print("Executing select query to test the data model as per given query") 

# Add header for output rows 

print("item_in_session\tartist_name\tsong_title\tfirst_name\tlast_name")
res = session.execute("select item_in_session, artist, song_title, first_name, last_name from user_session where user_id = 10 and session_id=182")
for r in res:
    print(str(r.item_in_session) +"\t" + r.artist + "\t" + r.song_title + "\t" + r.first_name +"\t" + r.last_name)

Table user_session created successfully!!
Loading data in cassandra
Data loading completed
Executing select query to test the data model as per given query
item_in_session	artist_name	song_title	first_name	last_name
0	Down To The Bone	Keep On Keepin' On	Sylvie	Cruz
1	Three Drives	Greece 2000	Sylvie	Cruz
2	Sebastien Tellier	Kilometer	Sylvie	Cruz
3	Lonnie Gordon	Catch You Baby (Steve Pitron & Max Sanna Radio Edit)	Sylvie	Cruz


###### In order to find details for a particular song we need to create a table with primary key song & will sort by userId by setting this as its clustering column.

######## Lets call the table user_history with column first_name, last_name with the aforesaid primary key & clustering columns.

In [12]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS user_history (song_title Text, user_id int, first_name Text, last_name Text, PRIMARY KEY (song_title, user_id))""")
    print("Table user_history created successfully!!")
except Exception as ex:
    print(f"Error creating table {str(ex)}")
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    count = 0
    print("Loading data in cassandra")
    for line in csvreader:
        query = "Insert into user_history (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s,%s,%s,%s)"
        artist, first_name,gender,item_in_session,last_name,length,level,location,session_id,song,user_id = line
        try:
            session.execute(query, (song, int(user_id), first_name , last_name))
            count += 1
        except Exception as ex:
            count += 1
            print(f"Error inserting row #{count} {str(ex)}")
print("Data loading completed")
print("Executing select query to test the data model as per given query")
res = session.execute("select first_name, last_name from user_history where song_title='All Hands Against His Own'")
# Add header row for the output rows
print("first_name\tlast_name")
for r in res:
    print(r.first_name +"\t" + r.last_name)

Table user_history created successfully!!
Loading data in cassandra
Data loading completed
Executing select query to test the data model as per given query
first_name	last_name
Jacqueline	Lynch
Tegan	Levine
Sara	Johnson


### Drop the tables before closing out the sessions

In [13]:
session.execute("DROP TABLE session_song")
session.execute("DROP TABLE user_session")
session.execute("DROP TABLE user_history")
session.execute("DROP keyspace event")

### Close the session and cluster connection

In [14]:
session.shutdown()
cluster.shutdown()